In [22]:
!python main.py experiments/general.cfg problems/hhcrsp/hhcrsp_5_4.cfg variants/hhcrsp.cfg -v

--------------DESCRIPTION HHCRSP-----------------------------------
n = 5, v = 4  
matrixQ: [[1, 0, 1, 1], [1, 1, 1, 0], [0, 1, 1, 0], [0, 1, 1, 1], [1, 0, 0, 1]]
matrixD: [[0, 4, 6, 6, 6], [7, 0, 7, 7, 6], [1, 6, 0, 1, 3], [6, 2, 1, 0, 2], [2, 7, 7, 7, 0]]
--------
activity 1: 1 3 4 
	 p = 1  
	 s = 16 --> e = 26 
activity 2: 1 2 3 
	 p = 8  
	 s = 4 --> e = 6 
activity 3: 2 3 
	 p = 2  
	 s = 10 --> e = 19 
activity 4: 2 3 4 
	 p = 7  
	 s = 8 --> e = 13 
activity 5: 1 4 
	 p = 6  
	 s = 9 --> e = 13 
--------
shift 1 -- u = 9 
shift 2 -- u = 20 
shift 3 -- u = 23 
shift 4 -- u = 15 
weights: 
	 w_x = 0.500000 , w_y = 0.250000 , w_z = 0.250000 , w_dependency = 0.500000
------------------------------------------------------------------
------------------------------------------------------------------
------------------------------------------------------------------
